In [1]:
import re
import os
import pandas as pd
import seaborn as sns

In [2]:
folder = os.path.join(os.getcwd(),'..','data','korpus')
subjects = ['Academic','Culture','European','Law','News',
            'Opinion','Parliament','Religion','Sport']

## Removing HTML tags from <i>korpus</i>

In [ ]:
expr = re.compile('<.*?>') 

def clean(html):
  html_clean = re.sub(expr, '', html)
  return html_clean

In [ ]:
for sbj in subjects:
    path = os.path.join(folder,sbj)
    
    for filename in os.listdir(path):
        filepath = os.path.join(path,filename)
        
        with open(filepath,'r', encoding='utf-8') as f:
            print(filename)
            text = f.read()
        
        with open(filepath,'w', encoding='utf-8') as f:
            f.write(clean(text))
    
    

## Turning text file into dataset

In [ ]:
for sbj in subjects:
    data = []
    path = os.path.join(folder,sbj)
    
    for filename in os.listdir(path):
        filepath = os.path.join(path,filename)
        
        with open(filepath,'r', encoding='utf-8') as f:
            print(filename)
            lines = f.readlines()
            
            for l in lines:
                d = l.split('\t')
                
                if len(d) > 1:
                    d[-1] = d[-1][:-1]
                    data.append(d)
                    
    df = pd.DataFrame(data, columns=['Word','POS','Lemma','Root'])
    df.to_csv(os.path.join(path,sbj+'.csv'), index=False)

## Joining individual csv files

We will create 2 versions of korpus.csv. One will simply be all the csv files together while the other will read an equal number of bytes from each file. This is so the frequency counts won't be biased based on the subject of the texts.

In [107]:
with open(os.path.join(folder,'korpus.csv'), 'w', encoding='utf-8') as f1:
    for sbj in subjects:
        with open(os.path.join(folder,sbj,sbj+'.csv'), 'r', encoding='utf-8') as f2:
            f1.write('\n')
            f1.write(f2.read())
            print(f'{sbj} Finished')

print('All Finished')


min_size = min([os.path.getsize(os.path.join(folder,sbj,sbj+'.csv')) for sbj in subjects])

#Only read min_length lines from each csv file. 

with open(os.path.join(folder,'norm_korpus.csv'), 'w', encoding='utf-8') as f1:
    for sbj in subjects:
        with open(os.path.join(folder,sbj,sbj+'.csv'), 'r', encoding='utf-8') as f2:    
            f1.write('\n')
            f1.write(f2.read(min_size))
            print(f'{sbj} Finished')

print('All Finished')


Academic Finished
Culture Finished
European Finished
Law Finished
News Finished
Opinion Finished
Parliament Finished
Religion Finished
Sport Finished
All Finished


## Exploring Dataset

In [108]:
%%time

#30 million rows
df = pd.read_csv(os.path.join(folder,'korpus.csv'),
                 usecols=["Word","POS"],
                 dtype={"Word": "U","POS": "S"},
                 nrows = 30000000)

df_norm = pd.read_csv(os.path.join(folder,'norm_korpus.csv'),
                      usecols=["Word","POS"],
                      dtype={"Word": "U","POS": "S"})

df.head(10)

Wall time: 10.6 s


,Word,POS
0,L-,DEF
1,għan,NOUN
2,prinċipali,ADJ
3,ta',GEN
4,Conectando,NOUN-PROP
5,Mundos,NOUN-PROP
6,(,X-PUN
7,Malta,NOUN-PROP
8,),X-PUN
9,(,X-PUN


### Cleaning

In [109]:
df['POS'].unique()

array(['DEF', 'NOUN', 'ADJ', 'GEN', 'NOUN-PROP', 'X-PUN', 'X-ABV', 'PREP',
       'CONJ-CORD', 'PART-PASS', 'PREP-DEF', 'PRON-PERS', 'COMP', 'VERB',
       'LIL-DEF', 'CONJ-SUB', 'KIEN', 'GEN-PRON', 'ADV', 'VERB-PSEU',
       'NEG', 'GEN-DEF', 'QUAN', 'PRON-DEM', 'X-DIG', 'PRON-INT', 'FOC',
       'PREP-PRON', 'NUM-WHD', 'LIL', 'NUM-CRD', 'X-ENG', 'X-FOR', 'PROG',
       'INT', 'X-BOR', 'PRON-PERS-NEG', 'LIL-PRON', 'PRON-INDEF',
       'PRON-DEM-DEF', 'NUM-ORD', 'HEMM', 'PRON-REF', 'PART-ACT', 'FUT',
       'NUM-FRC', 'PRON-REC', 'POS'], dtype=object)

In [112]:
%%time
#Maltese Tagset: https://mlrs.research.um.edu.mt/resources/malti03/tagset30.html

df = df.drop(df[df['POS']=='X-PUN'].index) #Punctuation
df = df.drop(df[df['POS']=='X-DIG'].index) #Digits
df = df.drop(df[df['POS']=='X-ENG'].index) #English
df = df.drop(df[df['POS']=='X-FOR'].index) #Foreign
df = df.drop(df[df['POS']=='X-ABV'].index) #Abbreviations
df = df.drop(df[df['POS']=='X-BOR'].index) #Gibberish
df = df.drop(df[df['POS']=='INT'].index)   #Interjections


df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-PUN'].index) #Punctuation
df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-DIG'].index) #Digits
df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-ENG'].index) #English
df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-FOR'].index) #Foreign
df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-ABV'].index) #Abbreviations
df_norm = df_norm.drop(df_norm[df_norm['POS']=='X-BOR'].index) #Gibberish
df_norm = df_norm.drop(df_norm[df_norm['POS']=='INT'].index)   #Interjections


Wall time: 2.55 s


In [117]:
%%time
df.to_csv(os.path.join(folder,'korpus_clean.csv'), index=False)
df_norm.to_csv(os.path.join(folder,'norm_korpus_clean.csv'), index=False)

Wall time: 4.01 s


In [15]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

missing_value_df

,column_name,percent_missing
Word,Word,0.000764
POS,POS,0.000000


## Frequency Counts

In [3]:
df = pd.read_csv(os.path.join(folder, "korpus_clean.csv"),
                 usecols=["Word","POS"],
                 dtype={"Word": "U","POS": "S"})

df_norm = pd.read_csv(os.path.join(folder, "norm_korpus_clean.csv"),
                 usecols=["Word","POS"],
                 dtype={"Word": "U","POS": "S"})


In [4]:
#Get all unique words
df_frequency = df.value_counts().to_frame()[0].reset_index()
df_frequency.columns = ['Word','POS','Frequency']


df_normal_frequency = df_norm.value_counts().to_frame()[0].reset_index()
df_normal_frequency.columns = ['Word','POS','Frequency']

df_frequency

,Word,POS,Frequency
0,ta',GEN,1136012
1,l-,DEF,1122350
2,li,COMP,741119
3,u,CONJ-CORD,603949
4,tal-,GEN-DEF,576924
...,...,...,...
289731,Francaise,NOUN-PROP,1
289732,ibridiżazzjoni,NOUN,1
289733,Franaġisku,NOUN-PROP,1
289734,ibsin,ADJ,1


### Removing words with an occcurence of less than 100.

In [6]:
df_frequency = df_frequency[df_frequency['Frequency'] > 100] 
df_normal_frequency = df_normal_frequency[df_normal_frequency['Frequency'] > 100] 

df_frequency.to_csv(os.path.join(folder,'korpus_frequency.csv'),index=False)
df_normal_frequency.to_csv(os.path.join(folder,'norm_korpus_frequency.csv'), index=False)

### Viewing most common nouns, adjectives or verbs.

In [7]:
df_frequency[(df_frequency["POS"] == "NOUN")|
             (df_frequency["POS"] == "ADJ") |
             (df_frequency["POS"] == "VERB")]

,Word,POS,Frequency
27,Regolament,NOUN,86504
31,Artikolu,NOUN,80061
34,Kummissjoni,NOUN,76837
42,Membri,NOUN,63317
43,Komunità,NOUN,62249
...,...,...,...
11011,jibda',VERB,101
11012,grazzja,NOUN,101
11014,ngħid,VERB,101
11015,ħarġa,NOUN,101


In [8]:
df_normal_frequency[(df_normal_frequency["POS"] == "NOUN")|
                    (df_normal_frequency["POS"] == "ADJ") |
                    (df_normal_frequency["POS"] == "VERB")]

,Word,POS,Frequency
10,ta',VERB,19076
30,sena,NOUN,9759
37,f',NOUN,8457
41,oħra,ADJ,8097
66,Ministru,NOUN,5912
...,...,...,...
3533,kuntenti,ADJ,101
3534,gvernijiet,NOUN,101
3536,wasslet,VERB,101
3537,Preżenti,ADJ,101


Since <i> ta` </i> can be considered both as a genitive clause (of) and a verb (to give), its frequency count may be innacurate.